In [ ]:
import os 
import importlib

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.Waveform import Waveform
from waffles.np04_data.ProtoDUNE_HD_APA_maps import APA_map # apa map
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid # channel grid

import integral_computation_function as integral_module
from integral_computation_function import channel_integral_computation

import waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots as utils_plots_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *


from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak import MyAnaPeak

import waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW as MyAnaPeak_NEW_module


In [ ]:
# Reading full streaming data

run_folder = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/full_streaming/no_selection"
fname = "processed_np04hd_raw_run027361_0001_dataflow0_datawriter_0_20240622T004415.hdf5_structured.hdf5"
filepath = os.path.join(run_folder, fname)
FS_wfset = load_structured_waveformset(filepath)  # load HDF5 structured waveform set  

In [ ]:
# Plotting original full-streaming waveforms --> Negative signals

importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, peak_bool = False, peak_beam_bool = False)

In [ ]:
# Peak finding analysis for beam event selection 
importlib.reload(MyAnaPeak_NEW_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW import MyAnaPeak_NEW

analysis_label = 'peak_finding'
analysis_class= MyAnaPeak_NEW

ip = IPDict(baseline_limits= [0, 200],
            n_std = 5,
            peak_distance= 500,
            beam_timeoffset_limits = [-40, -20],
            signal_sign = "negative"
            )

_ = FS_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf}), overwrite=True)

print('done\n\n')

In [ ]:
# Plotting full-streaming waveforms with peak finding results --> Negative signals

importlib.reload(utils_plots_module) 
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, peak_bool = False, peak_beam_bool = True, analysis_label = analysis_label)

In [ ]:
# Creo nuovo wfset per fare prove sulla selezione della finistra di full-streaming

# Lo slice adcs funziona per selezionare gli adcs, ma non trasla le altre informazioni 
# Tutte le info della wf non vengono modificare (timestamp, daq_window_timestamp, run_number,record_number,endpoint,channel,trigger_type)
# Viene solo modificato starting_tick che di default è 0 di solito
# Per quanto riguarda il wfset dopo che ho fatto apply, gli attributi rimango uguali compreso points_per_wf 187520


# Da fare: trovare 1/3 dell'ampiezza del picco e selezionare la finestra intorno 
# Da valutare: conviene rimuovere da prima la baseline? probabilmente si (magari usa metodo julio)

In [ ]:
prova_wfset = load_structured_waveformset(filepath)

In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 2, show  = True, peak_bool = False, peak_beam_bool = False, analysis_label = analysis_label)

In [ ]:
# Function to remove baseline and reverse the signal (if needed --> good for find_peaks)

from waffles.data_classes.WaveformAdcs import WaveformAdcs
from waffles.Exceptions import GenerateExceptionMessage

def my_subtract_baseline(
        waveform: WaveformAdcs,
        baseline_analysis_label: str,
        inversion: bool
) -> None:
    """This method overwrites the adcs method of the given
    WaveformAdcs object, by subtracting its baseline.

    waveform: WaveformAdcs
        The waveform whose adcs will be modified
    baseline_analysis_label: str
        The baseline to subtract must be available 
        under the 'baseline' key of the result of the analysis
        whose label is given by this parameter, i.e. in
        waveform.analyses[analysis_label].result['baseline']
    inversion: bool
        If True, the baseline-subtracted signal is inverted.
        If False, the baseline-subtracted signal is not inverted.
    """

    try:
        baseline = waveform.analyses[baseline_analysis_label].result['baseline']

    except KeyError:
        raise Exception(
            GenerateExceptionMessage(
                1,
                "subtract_baseline()",
                f"The given waveform does not have the analysis"
                f" '{baseline_analysis_label}' in its analyses "
                "attribute, or it does, but the 'baseline' key "
                "is not present in its result."
            )
        )
    
    if inversion:
        waveform._WaveformAdcs__set_adcs(
            baseline - waveform.adcs
        )
    else:
        waveform._WaveformAdcs__set_adcs(
            waveform.adcs - baseline
        )

    return

In [ ]:
#Tolgo la baseline subito e inverto 

from waffles.np04_analysis.led_calibration.utils import compute_average_baseline_std #baseline computation
from waffles.utils.baseline.baseline_utils import subtract_baseline # baseline subtraction
from waffles.utils.baseline.WindowBaseliner import WindowBaseliner
from waffles.data_classes.StoreWfAna import StoreWfAna 


baseline_limits = [0, 200]
baseliner_std_cut = 3.
baseliner_type = 'mean'
baseline_analysis_label = 'baseliner'
null_baseline_analysis_label = 'null_baseliner'


# Baseline
baseliner_input_parameters = IPDict({'baseline_limits': baseline_limits, 'std_cut': baseliner_std_cut, 'type': baseliner_type})    
checks_kwargs = IPDict({'points_no': prova_wfset.points_per_wf})
_ = prova_wfset.analyse(baseline_analysis_label, WindowBaseliner, baseliner_input_parameters, checks_kwargs=checks_kwargs, overwrite=True)

# Compute average baseline std
average_baseline_std = compute_average_baseline_std(prova_wfset, baseline_analysis_label)

# Remove baseline
prova_wfset.apply(my_subtract_baseline, baseline_analysis_label, inversion = True, show_progress=False)

In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 5, show  = True, peak_bool = False, peak_beam_bool = False, analysis_label = analysis_label)

In [ ]:
# Goal: cut full-streaming waveforms around the peak 
importlib.reload(MyAnaPeak_NEW_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW import MyAnaPeak_NEW
analysis_class = MyAnaPeak_NEW
analysis_label = 'finding_peaks'

ip = IPDict(baseline_limits= [0, 200],
            n_std = 5,
            peak_distance= 500,
            beam_timeoffset_limits = [-40, -20],
            signal_sign = "positive"
            )

_ = prova_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf}), overwrite=True)


In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 2, show  = True, peak_bool = True, peak_beam_bool = True, analysis_label = analysis_label)

In [ ]:
def filter_beam_peaks_0(waveform: Waveform, analysis_label) -> bool:
    if len(waveform.analyses[analysis_label].result['beam_peak_index']) == 1:
        return True
    else:
        return False
    
prova_wfset = WaveformSet.from_filtered_WaveformSet(prova_wfset, filter_beam_peaks_0, analysis_label)


In [ ]:
# To cut full-streaming waveforms around the peak

# The waveform must have the beaseline around zero and the signal inverted (positive peaks)


from waffles.data_classes.WaveformAdcs import WaveformAdcs
from waffles.Exceptions import GenerateExceptionMessage
import numpy as np

def cut_full_streaming_window_and_inverting(
        waveform: WaveformAdcs,
        analysis_label: str,
        amplitude_threshold: float = 0.3,
        pre_peak_window: int = 200,
        window_timeticks_length: int = 1000 #check the template, to have the same length
) -> None:
    """This method overwrites the adcs method of the given
    WaveformAdcs object.

    waveform: WaveformAdcs
        The waveform whose adcs will be modified
    analysis_label: str
        Name of the analysis including the peak information
    amplitude_threshold: float
        Percentage of the peak amplitude to define the starting point for the cut 
    """

    try:
        peak_index = waveform.analyses[analysis_label].result['beam_peak_index']
        baseline = waveform.analyses[analysis_label].result['mean_baseline']
    except KeyError:
        raise Exception(
            GenerateExceptionMessage(
                1,
                "Problem finding 'beam_peak_index' or 'beam_peak_amplitude' "
            )
        )
    
    # if len(peak_index) != 1:
    #     print("⚠️ peak_index length != 1, skipping…")
    #     return
    # else:
    peak_index = waveform.analyses[analysis_label].result['beam_peak_index'][0]
    peak_amplitude = waveform.analyses[analysis_label].result['beam_peak_amplitude'][0]

    # Searching for the index associated to peak_amplitude * amplitude_threshold (% of the raising edge)
    amplitude_threshold_dac = peak_amplitude * amplitude_threshold

    window = waveform.adcs[peak_index-pre_peak_window : peak_index]
    idx_in_window = np.abs(window - amplitude_threshold_dac).argmin()
    idx = (peak_index - pre_peak_window) + idx_in_window

    print(idx)

    start = idx - pre_peak_window
    stop  = start + window_timeticks_length

    if start < 0:
        start = 0
        stop = window_timeticks_length

    if stop > len(waveform.adcs):
        stop = len(waveform.adcs)
        start = stop - window_timeticks_length

    # Ultimo controllo: la lunghezza DEVE essere esattamente quella
    if stop - start != window_timeticks_length:
        print("Skipping: cannot enforce fixed window length")

    # ---- PARTE 5: slicing garantito omogeneo
    waveform._Waveform__slice_adcs(start, stop)

    return 


In [ ]:

prova_wfset.apply(cut_full_streaming_window_and_inverting, analysis_label = analysis_label ,show_progress=False)

In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 3, show  = True, peak_bool = False, peak_beam_bool = True, analysis_label = analysis_label)

In [ ]:
# Altre cose utili di sotto

In [ ]:
print("\nFS_wfset waveformset:")
print("points_per_wf:", FS_wfset.points_per_wf)
print("runs:", FS_wfset.runs)
print("record_numbers:", FS_wfset.record_numbers)
print("available_channels:", FS_wfset.available_channels)
print("mean_adcs:", FS_wfset.mean_adcs)
print("mean_adcs_idcs:", FS_wfset.mean_adcs_idcs)

print("\nprova_wfset waveformset:")
print("points_per_wf:", prova_wfset.points_per_wf)
print("runs:", prova_wfset.runs)
print("record_numbers:", prova_wfset.record_numbers)
print("available_channels:", prova_wfset.available_channels)
print("mean_adcs:", prova_wfset.mean_adcs)
print("mean_adcs_idcs:", prova_wfset.mean_adcs_idcs)

print("\n --------------------------------------------- \n")
i=2
print("\nprova_wfset waveform:")
print("timestamp:", prova_wfset.waveforms[i].timestamp)
print("daq_window_timestamp:", prova_wfset.waveforms[i].daq_window_timestamp)
print("run_number:", prova_wfset.waveforms[i].run_number)
print("record_number:", prova_wfset.waveforms[i].record_number)
print("endpoint:", prova_wfset.waveforms[i].endpoint)
print("channel:", prova_wfset.waveforms[i].channel)
print("starting_tick:", prova_wfset.waveforms[i].starting_tick)
print("trigger_type:", prova_wfset.waveforms[i].trigger_type)

print("\n\FS_wfset waveform:")
print("timestamp:", FS_wfset.waveforms[i].timestamp)
print("daq_window_timestamp:", FS_wfset.waveforms[i].daq_window_timestamp)
print("run_number:", FS_wfset.waveforms[i].run_number)
print("record_number:", FS_wfset.waveforms[i].record_number)
print("endpoint:", FS_wfset.waveforms[i].endpoint)
print("channel:", FS_wfset.waveforms[i].channel)
print("starting_tick:", FS_wfset.waveforms[i].starting_tick)
print("trigger_type:", FS_wfset.waveforms[i].trigger_type)

In [ ]:
# single waveform plot

i=0

fig = go.Figure()

y = prova_wfset.waveforms[i].adcs 
x = np.arange(len(y))
    
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='lines',
    line=dict(width=0.5),
    showlegend=False))


fig.add_trace(go.Scatter(
    x=prova_wfset.waveforms[i].analyses[analysis_label].result['beam_peak_time'],
    y=prova_wfset.waveforms[i].analyses[analysis_label].result['beam_peak_amplitude'],
    mode='markers',
    marker=dict(color='blue', size=8, symbol='circle'),
    name='Beam peaks'
    ))

peak_index = prova_wfset.waveforms[i].analyses[analysis_label].result['beam_peak_index'][0]
peak_amplitude = prova_wfset.waveforms[i].analyses[analysis_label].result['beam_peak_amplitude'][0]
baseline = prova_wfset.waveforms[i].analyses[analysis_label].result['mean_baseline']

# bisogna trovare 1/3 dell'ampiezza
amplitude_threshold_dac = peak_amplitude *(0.1)
idx = (prova_wfset.waveforms[i].adcs[peak_index-200:peak_index] - amplitude_threshold_dac).argmin() + peak_index

window_pre_peak = 200
window = prova_wfset.waveforms[i].adcs[peak_index-window_pre_peak : peak_index]
idx_in_window = np.abs(window - amplitude_threshold_dac).argmin()

idx = (peak_index - window_pre_peak) + idx_in_window

window = waveform.adcs[pre_peak_window-pre_peak_window : peak_index]
    idx_in_window = np.abs(window - amplitude_threshold_dac).argmin()
    idx = (peak_index - pre_peak_window) + idx_in_window

    print(idx)

print(amplitude_threshold_dac)
print(peak_index)

fig.add_shape(
    type="line",
    x0=idx,
    x1=idx,
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="coral", width=2, dash="dash"),
    name=f"(x = {idx})"
)

fig.show()